In [1]:
import os
os.chdir("../..")

In [2]:
import convokit

In [3]:
from convokit import Corpus

In [4]:
fake_news_dir = '/sauna/fake-news'
# politics_dir = '/sauna/reddit_201810_raw/corpus/pokemontrades_banlist~-~politics/politics'
reddit_dir = '/sauna/fake-news/reddit-corpus'
donald_corpus = '/sauna/reddit_201810_raw/corpus/TheTwoBeerQueers~-~The_Donald/The_Donald/'

In [5]:
corpus = Corpus(filename=reddit_dir)
# corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_basic_predictive_valid_convos'))

In [6]:
threads = corpus.utterance_threads(include_root=False)

In [7]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=5, include_root=False)

In [10]:
from collections import defaultdict
thread_roots_by_self_post = defaultdict(list)
for top_level_comment, thread in threads.items():
    if len(thread) < 5: continue
    rt = thread[next(iter(thread))].root
    thread_roots_by_self_post[rt].append(top_level_comment)

In [12]:
def get_thread_depth(utts): # List of utts
    depth = defaultdict(int)
    for utt in utts:
        depth[utt.id] = depth[utt.reply_to] + 1
    return max(depth.values())

In [13]:
def get_thread_width(utts): # List of utts
    width = defaultdict(int)
    for utt in utts:
        width[utt.reply_to] += 1
    return max(width.values())

In [14]:
def get_interaction_specificity(utts):
    utts = list(utts)
    return get_thread_depth(utts) / get_thread_width(utts)

In [20]:
# first generate positive and negative examples based on task
import random
def generate_pos_neg(task: str, post_to_thread_obj, threads, thread_pfxs):
    pos, neg = [], []
    if task == "comment-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos = [root for root in thread_roots if len(threads[root]) >= 15]
            has_neg = [root for root in thread_roots if len(threads[root]) == 10]
            
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    elif task == "commenter-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for root in thread_roots:
                if len(set(c.user.name for c in threads[root].values())) >= \
                    len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                    has_pos.append(root)
                elif len(set(c.user.name for c in threads[root].values())) == \
                    len(set(c.user.name for c in thread_pfxs[root].values())):
                    has_neg.append(root)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    elif task == "graph-depth-growth":
        for post_id, thread_ids in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for thread_id in thread_ids:
                if len(threads[thread_id]) >= 20:
                    if get_thread_depth(threads[thread_id].values()) >= \
                        get_thread_depth(thread_pfxs[thread_id].values()) * 2:
                        has_pos.append(thread_id)
                    else:
                        has_neg.append(thread_id)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))   
    elif task == "graph-width-growth":
        for post_id, thread_ids in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for thread_id in thread_ids:
                if len(threads[thread_id]) >= 20:
                    if get_thread_width(threads[thread_id].values()) >= \
                        get_thread_width(thread_pfxs[thread_id].values()) * 2:
                        has_pos.append(thread_id)
                    else:
                        has_neg.append(thread_id)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg)) 
    elif task == 'interaction-specificity':
        for post_id, thread_ids in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for thread_id in thread_ids:
                if len(threads[thread_id]) >= 20:
                    if get_interaction_specificity(threads[thread_id].values()) >= \
                        get_interaction_specificity(thread_pfxs[thread_id].values()) * 1.8:
                        has_pos.append(thread_id)
                    else:
                        has_neg.append(thread_id)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg)) 
    print("- {} positive, {} negative pts for {} task".format(len(pos), len(neg), task))
    
    return pos, neg

In [16]:
pos_comment_growth, neg_comment_growth = generate_pos_neg("comment-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 1827 positive, 1827 negative pts for comment-growth task


In [21]:
pos_commenter_growth, neg_commenter_growth = generate_pos_neg("commenter-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 769 positive, 769 negative pts for commenter-growth task


In [22]:
pos_depth_growth, neg_depth_growth = generate_pos_neg("graph-depth-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 626 positive, 626 negative pts for graph-depth-growth task


In [23]:
pos_width_growth, neg_width_growth = generate_pos_neg("graph-width-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 619 positive, 619 negative pts for graph-width-growth task


In [24]:
pos_int_spec, neg_int_spec = generate_pos_neg("interaction-specificity",
                                              thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 631 positive, 631 negative pts for interaction-specificity task


In [67]:
# import json
# with open('thread_ids_growth.json', 'w') as f:
#     ids = {"pos-comment-growth": pos_comment_growth,
#            "neg-comment-growth": neg_comment_growth,
#            "pos-commenter-growth": pos_commenter_growth,
#            "neg-commenter-growth": neg_commenter_growth
#           }
#     json.dump(ids, f)

In [13]:
# import json
# with open('thread_ids_growth.json', 'r') as f:
#     ids = json.load(f)
    
# pos_comment_growth = ids['pos-comment-growth']
# neg_comment_growth = ids['neg-comment-growth']
# pos_commenter_growth = ids['pos-commenter-growth']
# neg_commenter_growth = ids['neg-commenter-growth']

In [14]:
thread_to_convo = {thread_id: convo_id for convo_id, thread_ids in thread_roots_by_self_post.items() for thread_id in thread_ids}

In [20]:
commenter_growth_convos = set()
for thread_id in pos_commenter_growth:
    commenter_growth_convos.add(thread_to_convo[thread_id])

In [21]:
comment_growth_convos = set()
for thread_id in pos_comment_growth:
    comment_growth_convos.add(thread_to_convo[thread_id])

In [23]:
depth_growth_convos = set()
for thread_id in pos_depth_growth:
    depth_growth_convos.add(thread_to_convo[thread_id])

In [24]:
width_growth_convos = set()
for thread_id in pos_width_growth:
    width_growth_convos.add(thread_to_convo[thread_id])

In [25]:
paired_convos = comment_growth_convos.union(commenter_growth_convos).union(depth_growth_convos).union(width_growth_convos)

In [26]:
corpus.filter_conversations_by(lambda convo: convo.id in paired_convos)

In [27]:
# corpus.dump('donald_basic_predict_2', base_path=fake_news_dir)

In [25]:
hc = convokit.HyperConvo(prefix_len=5, min_thread_len=5, include_root=False)
orig_hc = convokit.HyperConvo_0(prefix_len=5, min_thread_len=5, include_root=False)


In [26]:
import pandas as pd

In [ ]:
hyperconvo_feats = orig_hc.retrieve_feats(corpus, prefix_len=10, min_thread_len=10, include_root=False)

/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo_orig/hyperconvo_o.py:177: RuntimeWarning: invalid value encountered in long_scalars
  "norm.max": lambda l: np.max(l) / np.sum(l),
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo_orig/hyperconvo_o.py:182: RuntimeWarning: invalid value encountered in long_scalars
  if len(l) > 1 else np.nan,
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2614: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo_orig/hyperc

In [ ]:
motif_feats = hc.retrieve_feats(corpus)

In [ ]:
motif_feats_df = pd.DataFrame.from_dict(motif_feats, orient='index')
motif_feat_names = list(motif_feats_df.columns)

In [ ]:
hyperconv_df = pd.DataFrame.from_dict(hyperconvo_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [ ]:
path_stats = hc.retrieve_motif_pathway_stats(corpus)
path_stats_df = pd.DataFrame.from_dict(path_stats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns

path_stats_enum_df = pd.DataFrame()

for path_stat in columns:
    path_stats_enum_df['is-present[{}]'.format(path_stat)] = path_stats_df[path_stat] > 0
    path_stats_enum_df['count[{}]'.format(path_stat)] = path_stats_df[path_stat]

path_feat_names = list(path_stats_enum_df.columns)

In [ ]:
def get_num_users(thread):
    return len(set(utt.user.name for utt in thread.values()))

thread_to_usercount = dict()
for thread_id in thread_pfxs:
    if len(thread_pfxs[thread_id]) < 10: continue
    thread_to_usercount[thread_id] = {"num_users": get_num_users(thread_pfxs[thread_id])}

In [ ]:
num_users_df = pd.DataFrame.from_dict(thread_to_usercount, orient='index')
num_users_feat = ['num_users']

In [ ]:
feats_df = pd.concat([hyperconv_df, motif_feats_df, path_stats_enum_df, num_users_df], axis=1)

In [28]:
# threads = {k: v for k, v in corpus.utterance_threads(include_root=False).items() if k in valid_threads}

In [29]:
# Use only the first 10 comments in each thread
# thread_pfxs = {k: v for k, v in corpus.utterance_threads(prefix_len=10, include_root=False).items() if k in valid_threads}

In [ ]:
feats_df = feats_df.astype('float64')

In [ ]:
feats_df = feats_df.fillna(-1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import cross_val_score
import numpy as np

for task in ["comment-growth", "commenter-growth", "graph-depth-growth", "graph-width-growth", "interaction-specificity"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    if task == "comment-growth":
        pos, neg = pos_comment_growth, neg_comment_growth
    elif task == "commenter-growth":
        pos, neg = pos_commenter_growth, neg_commenter_growth
    elif task == "graph-depth-growth":
        pos, neg = pos_depth_growth, neg_depth_growth
    elif task == "graph-width-growth":
        pos, neg = pos_width_growth, neg_width_growth
    elif task == "interaction-specificity":
        pos, neg = pos_int_spec, neg_int_spec
        
#     pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)
    for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (motif_feat_names, "motif"),
                        (path_feat_names, "motifpaths"),
                        (motif_feat_names + path_feat_names, "motif-all"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names, "hyperconvo-motifall"),
                        (num_users_feat, "usercount"),
#                         (hyperconv_feat_names + num_users_feat, "hyperconv-usercount"),
#                         (motif_feat_names + num_users_feat, "motif-usercount"),
#                         (path_feat_names + num_users_feat, "motifpaths-usercount"),
#                         (motif_feat_names + path_feat_names + num_users_feat, "motifsall+usercount"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names + num_users_feat, "hyperconvo-motifall+usercount"),
                       ]:
        clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      
        loo = LeaveOneOut()
        pp = convokit.PairedPrediction()
        X, y = pp._generate_paired_X_y(feats_df[feature_set], pos, neg)

#         clf.fit(X, y)
#         clf.score(X, y)
#         print(X.shape)
#         print(X[0])
#         print(y.shape)
        scores = cross_val_score(clf, X, y, cv=loo)
        print("- {}, cv_accuracy: {:.4f}".format(name, scores.mean()))


#         print("Feature set: {}".format(name))
#         pp.fit_predict(feats_df[feature_set], pos, neg, test_size=0.2)
#         pp.print_extreme_coefs(feature_set, num_features=5)